In [1]:
import pyhf
import json
import copy
import jsonpatch
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Download the Model


In [2]:
!curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2  | tar -O -xzv RegionA/BkgOnly.json > lhood.json
!curl -sL https://doi.org/10.17182/hepdata.89408.v1/r2  | tar -O -xzv RegionA/patch.sbottom_750_745_60.json > patch.json

x RegionA/BkgOnly.json
x RegionA/patch.sbottom_750_745_60.json


## Helper Functions


In [3]:
def make_model(channel_list):
    spec = json.load(open("lhood.json"))
    patch = json.load(open("patch.json"))
    spec = jsonpatch.apply_patch(spec, patch)
    spec["channels"] = [c for c in spec["channels"] if c["name"] in channel_list]

    ws = pyhf.Workspace(spec)
    model = ws.model(
        measurement_name="NormalMeasurement",
        modifier_settings={
            "normsys": {"interpcode": "code4"},
            "histosys": {"interpcode": "code4p"},
        },
    )
    data = ws.data(model)
    return ws, model, data

In [4]:
def fitresults(constraints=None):
    _, model, data = make_model(["CRtt_meff"])

    pyhf.set_backend("numpy", pyhf.optimize.minuit_optimizer(verbose=True))

    constraints = constraints or []
    init_pars = model.config.suggested_init()
    fixed_params = model.config.suggested_fixed()

    for idx, fixed_val in constraints:
        init_pars[idx] = fixed_val
        fixed_params[idx] = True

    lumi_parslice = model.config.par_slice('lumi')
    result = pyhf.infer.mle.fit(
        data,
        model,
        init_pars=init_pars,
        fixed_params=fixed_params,
        return_uncertainties=True,
    )
    bestfit = result[:, 0]
    errors = result[:, 1]
    return model, data, bestfit, errors

## Calculate per-parameter Impact


In [5]:
def calc_impact(idx, b, e, i, width, poi_index):
    _, _, bb, ee = fitresults([(idx, b + e)])
    poi_up_post = bb[poi_index]

    _, _, bb, ee = fitresults([(idx, b - e)])
    poi_dn_post = bb[poi_index]

    print('b,w', b, width)
    _, _, bb, ee = fitresults([(idx, b + width)])
    poi_up_pre = bb[poi_index]

    _, _, bb, ee = fitresults([(idx, b - width)])
    poi_dn_pre = bb[poi_index]
    return np.asarray([poi_dn_post, poi_up_post, poi_dn_pre, poi_up_pre])

In [21]:
def get_impact_data():
    model, _, b, e = fitresults()
    widths = pyhf.tensorlib.concatenate(
        [
            model.config.param_set(k).width()
            if model.config.param_set(k).constrained
            else [None]
            for k in model.config.par_order
        ]
    )
    initv = pyhf.tensorlib.concatenate(
        [model.config.param_set(k).suggested_init for k in model.config.par_order]
    )
    labels = np.asarray(
        [
            f"{k}[{i:02}]" if model.config.param_set(k).n_parameters > 1 else k
            for k in model.config.par_order
            for i in (
                range(model.config.param_set(k).n_parameters)
                if model.config.param_set(k).n_parameters > 0
                else [None]
            )
        ]
    )
    poi_free = b[model.config.poi_index]
    return_impacts = []
    return_labels = []

    lumi_parslice = model.config.par_slice('lumi')

    for i, (label, width) in enumerate(zip(labels, widths)):
        if width is None:
            continue
        if i == model.config.poi_index:
            continue
        if i % 5 == 0:
            print(i)
        impct = calc_impact(i, b[i], e[i], initv[i], width, model.config.poi_index)
        return_impacts.append(impct - poi_free)
        return_labels.append(label)
    return np.asarray(return_impacts), np.array(return_labels)

In [22]:
impacts, labels = get_impact_data()

615467201 0.017
┌──────────────────────────────────┬──────────────────────────────────────┐
│ FCN = 99.06                      │       Nfcn = 2995 (2995 total)       │
│ EDM = 1.18e-05 (Goal: 0.0002)    │                                      │
├───────────────┬──────────────────┼──────────────────────────────────────┤
│ Valid Minimum │ Valid Parameters │       SOME Parameters at limit       │
├───────────────┴──────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│   Hesse ok    │  Has Covariance  │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌──────────────────────────────────┬──────────────────────────────────────┐
│ FCN = 99.06                      │       Nfcn = 2873 (2873 total)       │
│ EDM = 0.000185 (Goal: 0.0002)    │                                    

## Make the plot!


In [23]:
impcord = np.argsort(np.max(np.abs(impacts[:, :2]), axis=1))
simpacts = impacts[impcord]
slabels = labels[impcord]

In [20]:
np.array(labels)[impcord]

array(['MUON_EFF_BADMUON_SYS', 'EL_EFF_Trigger_TOTAL_1NPCOR_PLUS_UNCOR',
       'MUON_EFF_TrigSystUncertainty', 'MUON_EFF_BADMUON_STAT',
       'EL_EFF_TriggerEff_TOTAL_1NPCOR_PLUS_UNCOR',
       'MUON_EFF_TrigStatUncertainty', 'MUON_SAGITTA_RHO',
       'EL_EFF_ChargeIDSel_TOTAL_1NPCOR_PLUS_UNCOR', 'MUON_EFF_TTVA_SYS',
       'MUON_EFF_TTVA_STAT', 'MUON_EFF_ISO_STAT', 'MUON_EFF_RECO_STAT',
       'EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR', 'MUON_EFF_ISO_SYS',
       'ttZ_theory', 'JET_EtaIntercalibration_NonClosure_negEta',
       'JET_EtaIntercalibration_NonClosure_highE', 'MUON_SCALE',
       'JET_EtaIntercalibration_NonClosure_posEta', 'MUON_EFF_RECO_SYS',
       'EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR', 'EG_RESOLUTION_ALL',
       'MUON_ID', 'JET_JER_EffectiveNP_4', 'JET_JER_EffectiveNP_1',
       'JET_JER_EffectiveNP_3', 'JET_JER_EffectiveNP_2',
       'JET_JER_EffectiveNP_6', 'JET_JER_EffectiveNP_5',
       'JET_JER_EffectiveNP_7restTerm', 'MET_SoftTrk_ResoPerp',
       'MET_SoftTrk_ResoP

In [ ]:
plt.barh(
    range(len(simpacts)),
    np.asarray(simpacts)[:, 0],
    alpha=0.75,
    linestyle="dashed",
    facecolor="r",
)
plt.barh(
    range(len(simpacts)),
    np.asarray(simpacts)[:, 1],
    alpha=0.75,
    linestyle="dashed",
    facecolor="b",
)
plt.barh(
    range(len(simpacts)),
    np.asarray(simpacts)[:, 2],
    alpha=0.75,
    linestyle="dashed",
    fill=None,
    edgecolor="r",
)
plt.barh(
    range(len(simpacts)),
    np.asarray(simpacts)[:, 3],
    alpha=0.75,
    linestyle="dashed",
    fill=None,
    edgecolor="b",
)
plt.xlim(-0.001, 0.001)
plt.ylim(-0.5, len(simpacts) - 0.5)
plt.gcf().set_size_inches(4, 17.5)
plt.title("Δµ")
plt.yticks(range(len(slabels)), slabels)
plt.grid();